In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.dirname(os.getcwd()))

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import tensorflow as tf
import keras_tuner as kt

from src.helper import create_and_prepare_dataset

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is using CPU")

TensorFlow is using CPU


In [3]:
PROCESSED_DATA_DIR = "../data/processed"
TRAINING_DATA_DIR = "../data/split/train"
VALIDATION_DATA_DIR = "../data/split/val"
TEST_DATA_DIR = "../data/split/test"

BATCH_SIZE = 5

In [4]:
training_dataset = create_and_prepare_dataset(TRAINING_DATA_DIR, BATCH_SIZE, True)
validation_dataset = create_and_prepare_dataset(VALIDATION_DATA_DIR, BATCH_SIZE)
test_dataset = create_and_prepare_dataset(TEST_DATA_DIR, BATCH_SIZE)

In [5]:
class MyHyperModel(kt.HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        for i in range(hp.Int('num_conv_layers', 1, 3)):
            model.add(tf.keras.layers.Conv1D(
                filters=hp.Int('filters_' + str(i), min_value=32, max_value=256, step=32),
                kernel_size=hp.Int('kernel_size_' + str(i), min_value=2, max_value=3),
                activation='relu',
                padding='same'
            ))
            model.add(tf.keras.layers.MaxPooling1D(
                pool_size=hp.Int('pool_size_' + str(i), min_value=2, max_value=2)
            ))

        model.add(tf.keras.layers.Flatten())

        for i in range(hp.Int('num_dense_layers', 1, 3)):
            model.add(tf.keras.layers.Dense(
                units=hp.Int('dense_units_' + str(i), min_value=32, max_value=1024, step=32),
                activation='relu'
            ))

        model.add(tf.keras.layers.Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

In [ ]:
hypermodel = MyHyperModel(input_shape=(21,3), num_classes=7)

tuner = kt.Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=40,
    hyperband_iterations=6,
    directory='../hyperband',
    project_name='layers'
)

tuner.search_space_summary()

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=4,
        restore_best_weights=True
    )

tuner.search(training_dataset, 
             epochs=20, 
             steps_per_epoch=None,
             validation_data=validation_dataset,
             validation_steps=None,
             callbacks=[early_stopping])

tuner.results_summary()

In [7]:
best_tuning_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hyperparameters.values)
print(best_tuning_model.summary())

val_loss, val_accuracy = best_tuning_model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")
print(f"Validation loss: {val_loss}")

{'num_conv_layers': 3, 'filters_0': 96, 'kernel_size_0': 3, 'pool_size_0': 2, 'num_dense_layers': 1, 'dense_units_0': 1024, 'filters_1': 224, 'kernel_size_1': 2, 'pool_size_1': 2, 'filters_2': 160, 'kernel_size_2': 3, 'pool_size_2': 2, 'dense_units_1': 352, 'dense_units_2': 576, 'tuner/epochs': 40, 'tuner/initial_epoch': 14, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0067'}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 21, 96)         │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 10, 96)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 10, 224)        │        43,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 5, 224)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 5, 160)         │       107,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 2, 160)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         7,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 487,751 (1.86 MB)

 Trainable params: 487,751 (1.86 MB)

 Non-trainable params: 0 (0.00 B)

None
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9231 - loss: 0.2370
Validation accuracy: 0.9285714030265808
Validation loss: 0.26872092485427856
